# Deepfake Detection Pipeline - Google Colab

Complete end-to-end deepfake detection system with training, evaluation, and inference.

**Features:**
- Train on FaceForensics++ dataset
- Video-level predictions with frame aggregation
- GPU acceleration (use Runtime > Change runtime type > GPU)
- Pre-trained model inference
- Visualization and metrics

**Runtime Recommendation:** GPU (T4 or better) for faster training

## 1. Setup and Installation

In [ ]:
# Install dependencies
!pip install -q torch torchvision timm
!pip install -q opencv-python-headless facenet-pytorch
!pip install -q scikit-learn pandas matplotlib seaborn
!pip install -q tqdm pyyaml

print("✓ Installation complete!")

In [ ]:
# Check GPU availability
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Training will be slow. Enable GPU: Runtime > Change runtime type > GPU")

In [ ]:
# Clone the repository (or upload your files)
!git clone https://github.com/YOUR_USERNAME/deepfake-detector.git
%cd deepfake-detector

# Or upload your local files:
# from google.colab import files
# uploaded = files.upload()  # Upload your project files

## 2. Data Preparation

Choose one of the following options:

### Option A: Use Sample Dataset (Quick Test)

In [ ]:
# Download a small sample dataset for quick testing
!mkdir -p data/sample_videos
!wget -O data/sample_videos/sample.zip https://YOUR_SAMPLE_DATASET_URL
!unzip -q data/sample_videos/sample.zip -d data/sample_videos/

print("✓ Sample data downloaded")

### Option B: Upload Your Own Videos

In [ ]:
from google.colab import files
import os

# Upload videos
print("Upload your videos (REAL and FAKE folders)...")
uploaded = files.upload()

# Organize uploaded files
!mkdir -p data/raw_videos/train/REAL
!mkdir -p data/raw_videos/train/FAKE

print("✓ Videos uploaded. Move them to data/raw_videos/train/REAL or FAKE folders")

### Option C: Mount Google Drive (Recommended for Large Datasets)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Link to your dataset in Google Drive
# DATASET_PATH = '/content/drive/MyDrive/deepfake_data'
# !ln -s $DATASET_PATH data/raw_videos

print("✓ Google Drive mounted")

## 3. Preprocessing Pipeline

In [ ]:
# Extract frames from videos
!python src/02_extract_frames.py \
    --input data/raw_videos \
    --output data/frames \
    --fps 3 \
    --max-frames 16

print("✓ Frame extraction complete")

In [ ]:
# Detect and crop faces
!python src/03_crop_faces.py \
    --input data/frames \
    --output data/frames \
    --size 224 \
    --device cuda

print("✓ Face cropping complete")

## 4. Training

In [ ]:
# Train the model
!python src/04_train.py \
    --config config.yaml \
    --data data/frames \
    --output outputs \
    --epochs 10 \
    --batch-size 32 \
    --device cuda

print("✓ Training complete")

In [ ]:
# View training history
import json
import matplotlib.pyplot as plt

with open('outputs/metrics/training_history.json', 'r') as f:
    history = json.load(f)

epochs = [h['epoch'] for h in history]
train_f1 = [h['train']['f1'] for h in history]
val_f1 = [h['val']['f1'] for h in history]

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1, 'o-', label='Train F1', linewidth=2)
plt.plot(epochs, val_f1, 's-', label='Val F1', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Best validation F1: {max(val_f1):.4f}")

## 5. Evaluation

In [ ]:
# Evaluate on test set
!python src/05_eval.py \
    --checkpoint outputs/models/best.pt \
    --data data/frames \
    --output outputs/metrics \
    --split test

print("✓ Evaluation complete")

In [ ]:
# View test results
import pandas as pd

df = pd.read_csv('outputs/metrics/test_report.csv', nrows=1)
print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Accuracy:  {df['accuracy'].values[0]:.2%}")
print(f"Precision: {df['precision'].values[0]:.2%}")
print(f"Recall:    {df['recall'].values[0]:.2%}")
print(f"F1 Score:  {df['f1'].values[0]:.2%}")
print("="*60)

In [ ]:
# Generate visualization plots
!python src/07_report_plots.py \
    --metrics-dir outputs/metrics \
    --output-dir outputs/plots

# Display plots
from IPython.display import Image, display

print("\n📊 Performance Metrics:")
display(Image('outputs/plots/prf_bar.png'))

print("\n📊 Confusion Matrix:")
display(Image('outputs/plots/confusion_matrix.png'))

print("\n📊 Training Curves:")
display(Image('outputs/plots/training_curves.png'))

## 6. Inference on New Videos

In [ ]:
# Upload a video to test
from google.colab import files

print("Upload a video file to test...")
uploaded = files.upload()
test_video = list(uploaded.keys())[0]
print(f"✓ Uploaded: {test_video}")

In [ ]:
# Run inference
import torch
import cv2
import numpy as np
from torchvision import transforms
from facenet_pytorch import MTCNN
from PIL import Image
import sys
sys.path.append('src/utils')
from model import load_checkpoint

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, config = load_checkpoint('outputs/models/best.pt', device)
model.eval()

# Initialize face detector
mtcnn = MTCNN(keep_all=False, device=device)

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Extract frames
cap = cv2.VideoCapture(test_video)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = max(1, int(fps / 3))  # 3 fps
frame_count = 0
predictions = []

print(f"Processing {test_video}...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % frame_interval == 0:
        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect face
        face = mtcnn(frame_rgb)
        
        if face is not None:
            # Convert to PIL and apply transforms
            face_pil = transforms.ToPILImage()(face)
            face_tensor = transform(face_pil).unsqueeze(0).to(device)
            
            # Predict
            with torch.no_grad():
                output = model(face_tensor)
                prob = torch.sigmoid(output).item()
                predictions.append(prob)
    
    frame_count += 1

cap.release()

# Aggregate predictions
if predictions:
    avg_prob = np.mean(predictions)
    result = "FAKE" if avg_prob > 0.5 else "REAL"
    confidence = avg_prob if avg_prob > 0.5 else (1 - avg_prob)
    
    print("\n" + "="*60)
    print(f"PREDICTION: {result}")
    print(f"Confidence: {confidence:.2%}")
    print(f"Fake Probability: {avg_prob:.2%}")
    print(f"Frames Analyzed: {len(predictions)}")
    print("="*60)
else:
    print("⚠️ No faces detected in video")

## 7. Download Results

In [ ]:
# Download trained model
from google.colab import files

files.download('outputs/models/best.pt')
print("✓ Model downloaded")

In [ ]:
# Download all results as zip
!zip -r outputs.zip outputs/
files.download('outputs.zip')
print("✓ All results downloaded")

## 8. Save to Google Drive

In [ ]:
# Copy outputs to Google Drive
!cp -r outputs /content/drive/MyDrive/deepfake_detector_outputs
print("✓ Results saved to Google Drive")

## Tips & Tricks

### For Faster Training:
1. Use GPU runtime (Runtime > Change runtime type > GPU)
2. Use T4 or A100 GPU if available
3. Increase batch size (if you have enough memory)
4. Reduce max_frames if dataset is large

### For Better Accuracy:
1. Train on full FaceForensics++ dataset (9,000+ videos)
2. Increase epochs to 15-20
3. Use data augmentation (already enabled)
4. Try different models: xception, efficientnet_b4

### Common Issues:
- **Out of Memory**: Reduce batch_size in training
- **No faces detected**: Videos must contain clear faces
- **Slow training**: Enable GPU runtime
- **Session timeout**: Save checkpoints regularly to Drive

### Next Steps:
1. Deploy as REST API using Flask/FastAPI
2. Build web interface with Gradio/Streamlit
3. Fine-tune on specific deepfake types
4. Test compression robustness (src/06_compress_test.py)